In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.utils import plot_model

In [2]:
import numpy as np
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

In [10]:
reweight_distribution([98,23,34],.04)

array([1.00000000e+00, 1.83039157e-16, 3.20864030e-12])

In [12]:
import tensorflow.keras as keras
import numpy as np
path = keras.utils.get_file('nietzsche.txt',
origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 1s 2us/step
Corpus length: 600893


In [54]:
maxlen =60
steps = 3
sentences = []
next_chars= []


In [55]:
text[0]

'p'

In [56]:
for i in range(0, len(text)-maxlen , steps):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

In [57]:
sentences

['preface\n\n\nsupposing that truth is a woman--what then? is the',
 'face\n\n\nsupposing that truth is a woman--what then? is there ',
 'e\n\n\nsupposing that truth is a woman--what then? is there not',
 '\nsupposing that truth is a woman--what then? is there not gr',
 'pposing that truth is a woman--what then? is there not groun',
 'sing that truth is a woman--what then? is there not ground\nf',
 'g that truth is a woman--what then? is there not ground\nfor ',
 'hat truth is a woman--what then? is there not ground\nfor sus',
 ' truth is a woman--what then? is there not ground\nfor suspec',
 'uth is a woman--what then? is there not ground\nfor suspectin',
 ' is a woman--what then? is there not ground\nfor suspecting t',
 ' a woman--what then? is there not ground\nfor suspecting that',
 'woman--what then? is there not ground\nfor suspecting that al',
 'an--what then? is there not ground\nfor suspecting that all p',
 '-what then? is there not ground\nfor suspecting that all phil',
 'at 

In [58]:
chars = sorted(list(set(text)))

In [59]:
x = np.zeros((len(sentences) , maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

In [60]:
x[0]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [61]:
char_indices = dict((char, chars.index(char)) for char in chars)


In [62]:
for i , sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t,char_indices[char]] = 1      
    y[i, char_indices[next_chars[i]]]= 1

In [43]:
len(char_indices)

57

In [63]:
from tensorflow.keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [64]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [65]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [68]:
import random
import sys
for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)
        print()

epoch 1
1565/1565 [==============================] - 148s 94ms/step - loss: 1.3871
--- Generating with seed: " each of his virtues must have been
separately acquired, nur"
------ temperature: 0.2
 each of his virtues must have been
separately acquired, nurically and the self-portentory and the self-present and the conscience of the present the propessial and the self-possession of the self-portentoment the conscience, the work the supposition of the self-present and the present sould the controlition of the self-power the proposition and the series of the self-portent the conscience and the self-porteness and believed the self-ported to the german 
------ temperature: 0.5
e self-porteness and believed the self-ported to the german instinct that is the dispositions and the states of the interpretation to the world is there have the philosophy that the individuals of the stupidity with the truth the slave, the spirits of the contempt of the spirit in the
world and them as the world is th

to hortiank by knowledge, does if this feeling over metak ryers henful.--such a in, and that downher, than what sole, at they appearate, enolunt and illets
with men is effect, moves we our ground itideowgeness music: devinonbest have
decession. in point
be noted socopined would hazent who detimuchs--as dene; it
windest pwe greate and other
hand the renovess instinct,
incumines, and o
epoch 5
1565/1565 [==============================] - 139s 89ms/step - loss: 1.3543
--- Generating with seed: "l,
genuine token of the german soul, which is at the same ti"
------ temperature: 0.2
l,
genuine token of the german soul, which is at the same time to the problem of the discourse of the sense of the stronger of the same the stronger the stronger to the same the consideration of the stronger and the stronger and the profound the strong the stronger the stronger to the stronger and soul, and in the development of the same and the strong the same and the stronger to the servile of the serve the disc

] sching of the priurimle. what. in the
felther are spalt for instances, and impressor who 
will". the
woild).
an mustom, steenching even for the
apppered, on
the
phil
------ temperature: 1.2
d).
an mustom, steenching even for the
apppered, on
the
philooly we cafe means to spirits, than the
palter for calling will in the potence
and sactity aidsulity wanto one some impromands the art of instrume religiom.
byinhilt--the
even compulsion of minlele, toose
consists assiforth to roupssh is the pleasure of the nordends
a night and aravined,
condition to ? echong heining and druxarder in the
facts of blyss,
is
free souls the pretencedy for  of hragar,
epoch 9
1290/1565 [=======================>......] - ETA: 24s - loss: 1.3246

KeyboardInterrupt: 